In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier

: 

In [ ]:
import pandas as pd

json_file_path = 'train_for_student.json'
df = pd.read_json(json_file_path)
df = df.T


: 

In [ ]:
df.head()
df["CONTEXT"]=df['Title']+". "+df["Abstract"]

: 

In [ ]:
df.drop(df.columns[0:2], axis=1, inplace=True)


: 

In [ ]:
df=df[['CONTEXT','Classes']]

: 

In [ ]:
df

: 

In [ ]:
y=df['Classes']

: 

In [ ]:
y

: 

In [ ]:
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(df['Classes'])
y

: 

In [ ]:
y[0]

: 

In [ ]:
multilabel.classes_

: 

In [ ]:
len(multilabel.classes_)

: 

In [ ]:
CLASSES_ARRANGE = ['CE', 'ENV', 'BME', 'PE', 'METAL', 'ME', 'EE',
               'CPE', 'OPTIC', 'NANO', 'CHE', 'MATENG', 'AGRI',
               'EDU', 'IE', 'SAFETY', 'MATH', 'MATSCI']

: 

In [ ]:
new_y=pd.DataFrame(y, columns=multilabel.classes_)

: 

In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['CONTEXT'], new_y, test_size=0.2, random_state=0)

: 

In [ ]:
f1_scorer = make_scorer(f1_score, average='weighted')

: 

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='word', max_features=1000)),
    ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
])

: 

In [ ]:
parameters = {
    'tfidf__max_df': (0.75, 0.85, 0.95),
    'tfidf__min_df': (0.01, 0.05, 0.1),
    'clf__estimator__C': (0.01, 0.1, 1, 10, 100,1000,10000),  # Increased range for C
}

: 

In [ ]:
# Grid search to find the best parameters for both the vectorizer and the classifier
grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, scoring=f1_scorer, verbose=2)
grid_search_tune.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

: 

In [ ]:
# Prediction & Evaluation
y_pred = grid_search_tune.predict(X_test)

: 

In [ ]:
def print_score(y_true, y_pred):
    print('F1 score: {:.2f}'.format(f1_score(y_true, y_pred, average='weighted')))
    print('Recall score: {:.2f}'.format(recall_score(y_true, y_pred, average='weighted')))
    # Add more metrics here if needed

print_score(y_test, y_pred)

: 

In [ ]:
json_file_path = 'test_for_student.json'
test_df = pd.read_json(json_file_path)
test_df = test_df.T
test_df["CONTEXT"]=test_df['Title']+'. '+test_df["Abstract"]
test_df.drop(test_df.columns[0:2], axis=1, inplace=True)

: 

In [ ]:
test_df.head()

: 

In [ ]:
test_predictions = grid_search_tune.predict(test_df['CONTEXT'])
print(test_predictions)

: 

In [ ]:
classes_array = multilabel.classes_.tolist()

: 

In [ ]:
columns = ['id']+classes_array
data = []

# Enumerate through predictions
for i, pred in enumerate(test_predictions):
    # Create a row with id and predictions
    row = ['{:03d}eval'.format(i+1)] + list(pred)
    data.append(row)

new_df = pd.DataFrame(data, columns=columns)
print(new_df)


: 

In [ ]:
# Rearrange the columns of the DataFrame
new_df = new_df[['id'] + CLASSES_ARRANGE]

# Print the rearranged DataFrame
print(new_df)

: 

In [ ]:
row_sums = new_df.iloc[:, 1:].sum(axis=1)
#print(row_sums)
# Check how many of these sums are equal to 0
num_rows_with_sum_zero = (row_sums == 0).sum()

print(num_rows_with_sum_zero,max(row_sums))

: 

In [ ]:

# Assuming 'new_df' is your DataFrame to be saved
path='kaggle_submission2.csv'
# Save DataFrame to CSV
new_df.to_csv(path, index=False)

print("DataFrame saved to 'kaggle_submission.csv'")

: 